### Basic global settings trials including ncluster, method and data used

In [1]:
import os
import json
import itertools
import pandas as pd
from functions import *
%set_env GUROBI_HOME=/usr/local/gurobi/linux64

env: GUROBI_HOME=/usr/local/gurobi/linux64


In [2]:
with open('config.json', 'r') as f:
    config_all = json.loads(f.read())

config = config_all['Texas_7_years']
inputs = config['inputs']
settings = config['settings']
print(inputs)
pv_all = np.genfromtxt(
    os.path.join(config['root'], config['pv_path']), delimiter=',', filling_values=0)
wind_all = np.genfromtxt(
    os.path.join(config['root'], config['wind_path']), delimiter=',', filling_values=0)
load_all = np.genfromtxt(
    os.path.join(config['root'], config['load_path']), delimiter=',', filling_values=0)  # Data load

{'c_bat': 200000, 'c_bat_power': 70000, 'c_pv': 1000000, 'c_wind': 1500000, 'c_gen_inv': 1000000, 'c_gen_a': 0.1, 'c_gen_b': 30, 'up_time': 6, 'down_time': 6, 'energy_cap': 200, 'power_cap': 200, 'gen_cap': 0, 'mdc': 50, 'effi': 0.9, 'rps': 0.8, 'note_2': 'This property is set to indicate what are above properties for model. Change with model parameters.', 'solver': 1, 'print_log': 0, 'mipgap': 0.0001, 'timelimit': 210240, 'note_3': 'This property is set to indicate what are above properties for solver. No need to change.'}


In [3]:
day_num = settings['day_num']
time_set = np.arange(24*day_num)  # Time horizon
if 'profile_id' in settings:
    profile_id = settings['profile_id']
    renewable = [wind_all[time_set, profile_id],pv_all[time_set, profile_id]]
else:
    # Denote that first profile is used and no other profile exists.
    profile_id = -1
    renewable = [wind_all[time_set], pv_all[time_set]]
load = load_all[time_set]*100  # Data extract

In [4]:
list_temp = itertools.product([100000,150000],[50000,30000],[500000,750000],[1000000,1250000])
backups = pd.DataFrame(columns=['c_bat','c_bat_power','c_pv','c_wind'],data=list_temp)
for backup in [0,1,2,3]:
    print("Index :" + str(backup))
    update = backups.loc[backup,:].to_dict()
    for item in ['c_bat','c_bat_power','c_pv','c_wind']:
        inputs[item] = update[item]
    #inputs['print_log'] = 1
    config['inputs'] = inputs
    """# Full time model solve
    start_time = time.time()
    m = time_model_solve(inputs, renewable, load)
    elapsed_time = time.time() - start_time
    results = dict()
    for v in m.component_objects(Var, active=True):
        var_object = getattr(m, str(v))
        results[str(v)] = []
        if var_object.is_indexed() and len(var_object) >= 1:
            for t in range(len(var_object)):
                results[str(v)].append(var_object[t].value)
        elif len(var_object) == 1:
            results[str(v)] = var_object.value

    # Benchmark calculation
    to_add = ['renewable_cap', 'N', 'max_energy', 'max_power', 'total_cost'] if inputs['gen_cap'] else [
    'renewable_cap', 'max_energy', 'max_power', 'total_cost']
    expected = {v: results[v] for v in to_add}
    expected['description'] = 'benchmark'
    expected['elapsed_time'] = elapsed_time
    for i in range(len(renewable)):
        expected['renewable_cap_{}'.format(i)] = results['renewable_cap'][i]
    pd.DataFrame([expected]).to_csv('/home/jupyter-zyh/Gnhe/benchmark/Linear/Texas/profile-{}-Int/expected/expected_{}_{}_1e-4.csv'.format(profile_id,day_num,backup))
    print("Expectation : Done.")
    inputs['print_log'] = 0
    config['inputs'] = inputs
    sim_features_df = sim_features(config,wind_all,pv_all,load_all)
    sim_features_df.to_csv('/home/jupyter-zyh/Gnhe/benchmark/Linear/Texas/profile-{}-Int/features/features_{}_{}_1e-4.csv'.format(profile_id,day_num,backup))
    print("Simulation : Done.")"""
    
    expected = pd.read_csv('/home/jupyter-zyh/Gnhe/benchmark/Linear/Texas/profile-1/expected/expected_2555_{}_1e-4.csv'.format(backup), index_col=0).loc[0,:].to_dict()
    sim_features_df = pd.read_csv('/home/jupyter-zyh/Gnhe/benchmark/Linear/Texas/profile-1/features/features_2555_{}_1e-4.csv'.format(backup),index_col=0)
    df = run_trials(config, wind_all, pv_all, load_all, expected, sim_features_df[['renewable_cap_0','renewable_cap_1','max_energy','max_power']])
    df.set_index('description').to_csv(
        config['save_root']+settings['data']+('_Int_' if inputs['gen_cap'] else '_')+('Renewable'+str(profile_id)+'_' if 'profile_id' in settings else '_')+str(settings['day_num'])+'Days_'+str(settings['period'])+'HrsPerPeriod_{}_30.csv'.format(backup))
print("All Done!")

Index :0
1th trial:  23.9145405292511
2th trial:  28.989708185195923
3th trial:  47.216710805892944
4th trial:  40.215611696243286
5th trial:  58.66111469268799
6th trial:  69.42632603645325
Index :1
1th trial:  24.706538915634155
2th trial:  28.771324634552002
3th trial:  38.87427544593811
4th trial:  46.125957012176514
5th trial:  56.8280553817749
6th trial:  58.69496536254883
Index :2
1th trial:  24.12559723854065
2th trial:  28.556776762008667
3th trial:  42.704293966293335
4th trial:  51.24927878379822
5th trial:  53.51633405685425
6th trial:  63.4648642539978
Index :3
1th trial:  24.73146367073059
2th trial:  29.151500701904297
3th trial:  42.765852212905884
4th trial:  49.6628303527832
5th trial:  57.140588998794556
6th trial:  71.84397649765015
All Done!
